In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

In [4]:
# Create optimus
op = Optimus()


                             ____        __  _                     
                            / __ \____  / /_(_)___ ___  __  _______
                           / / / / __ \/ __/ / __ `__ \/ / / / ___/
                          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
                          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
                              /_/                                  
                              
Transform and Roll out...
Just checking that all necessary environments vars are present...
-----
HADOOP_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\Program Files\Java\jdk1.8.0_181
Pyarrow Installed
-----
Starting or getting SparkSession and SparkContext...
Optimus successfully imported. Have fun :).


In [5]:
df = op.create.df([
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("second", "int", True),
                ("filter", StringType(), True)
            ],
            [
                ("  I like     fish  ", 1, "dog dog", "housé", 5 , "a"),
                ("    zombies", 2, "cat", "tv", 6, "b"),
                ("simpsons   cat lady", 2, "frog", "table", 7, "1"),
                (None, 3, "eagle", "glass", 8, "c")
                
            ])

df.table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
⸱⸱⸱⸱zombies,2,cat,tv,6,b
simpsons⸱⸱⸱cat⸱lady,2,frog,table,7,1
None,3,eagle,glass,8,c


In [6]:
df.dtypes

[('words', 'string'),
 ('num', 'int'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('second', 'int'),
 ('filter', 'string')]

## Append row
### Spark
Not available in Spark. You need to create a dataframe and the union to append a row

In [7]:
df.rows.append(["this is a word",2, "this is an animal", "this is a thing", 64, "this is a filter"]).table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
⸱⸱⸱⸱zombies,2,cat,tv,6,b
simpsons⸱⸱⸱cat⸱lady,2,frog,table,7,1
None,3,eagle,glass,8,c
this⸱is⸱a⸱word,2,this⸱is⸱an⸱animal,this⸱is⸱a⸱thing,64,this⸱is⸱a⸱filter


## Sort

### Sort columns asc(This is the default value)

In [34]:
df.rows.sort("animals").table()

2
(['animals'], 'asc')


words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱⸱⸱zombies,2,cat,tv,6,b
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
None,3,eagle,glass,8,c
simpsons⸱⸱⸱cat⸱lady,2,frog,table,7,1


In [38]:
df.rows.sort("animals", "desc").table()

(['animals'], 'desc')


words (string),num (int),animals (string),thing (string),second (int),filter (string)
simpsons⸱⸱⸱cat⸱lady,2,frog,table,7,1
None,3,eagle,glass,8,c
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
⸱⸱⸱⸱zombies,2,cat,tv,6,b


### Sort by multiples columns

In [39]:
df.rows.sort([("animals","desc"),("thing","asc")]).table()

('animals', 'desc')
('thing', 'asc')


words (string),num (int),animals (string),thing (string),second (int),filter (string)
simpsons⸱⸱⸱cat⸱lady,2,frog,table,7,1
None,3,eagle,glass,8,c
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
⸱⸱⸱⸱zombies,2,cat,tv,6,b


In [21]:
a = [("animals","desc"),("thing","asc")]
for c in a:
    print(c[0])

animals
thing


## Filter

In [9]:
df.rows.select(df["num"]==1).table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a


## Filter by type
### Spark 
Not available in Spark Vanilla.

In [10]:
df.rows.select_by_dtypes("filter", "integer").table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
simpsons⸱⸱⸱cat⸱lady,2,frog,table,7,1


## Drop 
### Spark
Drop by row not available in Spark Vanilla

In [11]:
df.rows.drop((df["num"]==2) | (df["second"]==5)).table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
None,3,eagle,glass,8,c


In [12]:
df.rows.drop_by_dtypes("filter", "int").table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
⸱⸱⸱⸱zombies,2,cat,tv,6,b
None,3,eagle,glass,8,c


### Drop by type

In [13]:
df.rows.drop_by_dtypes("filter", "integer").table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
⸱⸱⸱⸱zombies,2,cat,tv,6,b
None,3,eagle,glass,8,c


### Drop using an abstract UDF

In [14]:
from optimus.functions import abstract_udf as audf

def func_data_type(value, attr):
    return value >1


df.rows.drop(audf("num", func_data_type, "boolean")).table()

words (string),num (int),animals (string),thing (string),second (int),filter (string)
⸱⸱I⸱like⸱⸱⸱⸱⸱fish⸱⸱,1,dog⸱dog,housé,5,a
